In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [2]:
#********************************************************************************
# Import libraries
#********************************************************************************

"""
Import libraries from which to call functions. These are the R equivalent of packages
Libraries need to be installed first within the terminal using e.g. 'pip install dash'. 
Note that this should be a different terminal to that displaying the jupyter notebook outputs
"""

from statsmodels.tsa.ar_model import AutoReg # Library for forecasted values
from scipy import stats # Builds on NumPy, for further manipulation and visualisation
import numpy as np # Library for data analysis which can be called as 'np'
import pandas as pd  # library for data analysis which can be called as 'pd'
import plotly.express as px # Library for data visualisation called as 'px'
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
import traceback # standard interface to extract, format and print stack traces i.e. code reports
import requests  # library to handle requests
import dash # Library to generate dashboard
from jupyter_dash import JupyterDash # Library to generate dashboard
from dash import dcc # Components for generating dashboard
from dash import html # Components for generating dashboard
from dash.dependencies import Input, Output # Components for generating dashboard


## Importing Data

In [3]:
#********************************************************************************
# Import data
#********************************************************************************

# Disable scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)

"""
Define functions to import data sheet by sheet from the excel
Using a try block lets you test a block of code for errors or 'exceptions'
If the try block raises an error, the except block will be executed
"""

# Import data sheet for product identifier, lifespan information, mass information and source

def import_product_data(path):
    """
    :return:
    """
    try:
        # List of fields to retain from the imported file, with those not listed to be dropped
        fields = ['product_category(2)', 
                  # 'PRODCOM',
                  # 'HS6/CN6',
                  'Lifespan_EoL_lower_yr',
                  'Lifespan_EoL_upper_yr',
                  'Lifespan_EoL_average_yr',
                  'Mass_lower_kg',
                  'Mass_upper_kg',
                  'Mass_average_kg',
                  'Source',
                  'Source URL',
                  'Lifespan_Weibull_scale',
                  'Lifespan_Weibull_shape']

        # Read in the data file while keeping only selected fields
        Product_data = pd.read_excel(path, sheet_name = "Lookup")# usecols=fields)

        # Clear column headings of spaces and anything after brackets/parenthesis to make calling them easier
        Product_data.columns = Product_data.columns.str.replace(' ', '')
        
        # Filter to products in the product category 2 column
        Product_data = Product_data[Product_data.Product_type == 'Final good']
        
        return Product_data
    except Exception as e:
        print("Exception in import_product_data: {}".format(str(e)))
        print(traceback.format_exc())

# Import flow 

def import_domestic_flow(path):
    """
    :return:
    """
    try:
        # Read in the data file and sheet
        domestic_flow = pd.read_excel(path, sheet_name = "Flow_domestic")
        
        # Filter to number of items in the indicator column
        filter_list_flow_indicator = ['Volume (Number of items)']
        domestic_flow = domestic_flow[domestic_flow.Indicator.isin(filter_list_flow_indicator)]
        domestic_flow = domestic_flow.pivot(columns="Year", values="Value", index=["Product", "Flow", "Indicator"])
        domestic_flow = domestic_flow.fillna(0)
        
        return domestic_flow
    except Exception as e:
        print("Exception in import_product_data: {}".format(str(e)))
        print(traceback.format_exc())

# Import product composition data for treemap graph
        
def import_composition_data(path):
    """
    :return:
    """
    try:
        # Read in the data file
        Composition_data = pd.read_excel(path, sheet_name = "Composition")
        return Composition_data
    except Exception as e:
        print("Exception in import_product_data: {}".format(str(e)))
        print(traceback.format_exc())
        
# Import all data above      

def import_trade_flow(path_to_file):
    flow_trade = pd.read_excel("dataset.xlsx", sheet_name="Flow_trade")
    flow_trade = flow_trade[["Product", "Year", "Value", "Flow", "Indicator"]].pivot(index=["Product", "Flow"], columns=["Indicator", "Year"], values="Value")["Volume (Number of items)"]
    flow_trade = flow_trade.fillna(0)

    return flow_trade
        
def import_stock_data(path_to_file):
    storage_data = pd.read_excel(path_to_file, sheet_name="Stocks").iloc[3:]
    return storage_data

def import_data(path_to_file):
    """
    This functions combines the three functions above
    :return: outputs of the above import data functions as dataframes
    """
    return import_composition_data(path_to_file), import_product_data(path_to_file), import_domestic_flow(path_to_file) , import_trade_flow(path_to_file), import_stock_data(path_to_file)


# Define products list

def get_products_list(product_data, domestic_flow):
    """
    This functions combines product data and flow data.
    It takes all the unique products from product data and keeps only the ones where flow data is available
    :return: list of products
    """
    list_products = product_data["product_category(2)"].unique() # Take all the unique products from product data
    list_flow = domestic_flow.index.get_level_values(0).unique() # Take all the unique products from flow data
    
    return list(set(list_products) & set(list_flow))

composition_data, product_data, domestic_flow, flow_trade, storage_data = import_data("dataset.xlsx")

margins = dict(
    l=10,
    r=30,
    b=20,
    t=100)

## Sidebar

In [4]:
# *******************************************************************************
# Define Sidebar/Navbar
#********************************************************************************

# Set Navbar options 

NAVBAR_OBJECTS_MARGIN = "30px"

# Defining three attributes for the navbar
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "20rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# Define Sidebar Objects

# Drop down for product selection. 
navbar_product_dropdown = dcc.Dropdown(get_products_list(product_data, domestic_flow), 
                                       'Laptop', 
                                       id='navbar-product-dropdown', 
                                       style={"margin-bottom": "15px"})

# Slider for lifespan selection
navbar_lifespan_slider = dcc.Slider(min=-40,
                                    max=40,
                                    step=20,
                                    value=0,
                                    id='navbar-lifespan-slider',
                                    marks={
                                        i:str(i) + "%" for i in range(-40,41,20)
                                    })

# Slider for lifespan selection

MAX_YEAR, MIN_YEAR = 2050, 1980

navbar_interestwindow_slider = dcc.RangeSlider(min=1989, 
                                          max=2050, 
                                          step=5, 
                                          value=[2010,2050], 
                                          id='navbar-interestwindow-slider',
                                          marks={
                                            i:"'"+str(i)[2:] for i in range(MIN_YEAR, MAX_YEAR, 5) 
                                          })

weibull_scale = dcc.Slider(
    min=0,
    max=20,
    value=5,
    id="weibull-scale"
)
weibull_shape = dcc.Slider(
    min=0,
    max=4,
    value=1,
    id="weibull-shape"
)

# Adding the sidebar objects to the sidebar
sidebar = html.Div(
    [
        html.H2("Product simulator", className="display-4"),
        html.Hr(),
        dbc.Nav(
            [
               html.H6("Select"),
               navbar_product_dropdown,
                
               # html.H6("Lifespan Input Slider"),
               # navbar_lifespan_slider,
               # html.Div(style={"margin-bottom": NAVBAR_OBJECTS_MARGIN}),
                
               html.H6("Period of interest"),
               navbar_interestwindow_slider,
                html.Br(),

                
                html.H6("Weibull scale"),
                weibull_scale,
                html.Br(),
                html.H6("Weibull shape"),
                weibull_shape,
                html.Br(),

                
                html.H6(id="median-lifetime"),
                html.Br(),
                # html.Div(),
                dbc.Alert(html.H6(id="eol-lower", style={"textAlign":"center"}), color="secondary"),
                dbc.Alert(html.H6(id="eol-upper", style={"textAlign":"center"}), color="secondary"),
                #html.Br(),
                dbc.Alert(html.H6(id="eol-average", style={"textAlign":"center"}), color="secondary"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

html.Div(
    [
      dcc.Input(),
      dcc.Input(style={"margin-left": "15px"})
    ]
)


Div([Input(None), Input(style={'margin-left': '15px'})])

## Main App

In [5]:
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

In [6]:
def forecast(data):
    min_year = min(data.index)
    max_year = max(data.index)
    # idx = pd.date_range(str(min_year), str(max_year+1), freq="Y")
    # x = pd.Series(data.values)# , idx)
    # ar = AutoReg(x, 2)
    # res = ar.fit()
    # pred = res.forecast(MAX_YEAR-max_year)

    pred = pd.Series(data.iloc[-1] * np.ones(len(list(range(max_year+1, MAX_YEAR+1)))))
    pred.index = list(range(max_year+1, MAX_YEAR+1))


    return pred

In [7]:
def get_treemap(product):
    """
    buillds a treemap out of the composition data given the product name (e.g. 'Laptop')
    :return: The treemap plotly object
    """
    
    # Filtering the composition data for selected product from dropdown
    data = composition_data.loc[composition_data["Product"] == product] 
    data["Percentage"] = data["Percentage"]*100 # Multiplies the percentage value by 100 to compensate for the % symbol after values in the excel sheet
    
    # Defining the treemap to be generated from the data filtered above for the selected product
    tree_fig = px.treemap(
        data_frame=data, # The filtered data
        parents="Product", # The parent at which level the tree data has to be considered
        names="Material", # Name of the blocks in the treemap
        values="Percentage", # The values on whose basis the size of the box will be decided
        title="<b>Material composition</b>", # The title of the treemap
        )
    
    tree_fig.data[0].hovertemplate = '%{label}<br>%{value}%'
    
    # Updating the layout of the plot to make it fit within requirements
    tree_fig.update_layout(
        margin=margins,
        autosize=True,
        
        ## changed figure width from 800 to 500 so that the figure doesn't go too far to the right. ##
        # width=500,
        
        
        # height=300,
    )
    
    return tree_fig

In [8]:
def get_eol(product):
    """
    given a product, return the lower, upper and average End of Life
    """
    data = product_data.loc[product_data["product_category(2)"] == product]

    return data["Lifespan_EoL_lower_yr"], data["Lifespan_EoL_upper_yr"], data["Lifespan_EoL_average_yr"]

def get_flow(product="Laptop", var="Domestic", window:list = [2010,2050]):
    """
    This functions generates the barplot from flow data and cumulative values in a given time window
    :return: Bar plot and cumulative value 
    """
    
    # Select the data from flow data only for a given product and variable (import or domestic)
    data_domestic = domestic_flow.loc[product, "Domestic", "Volume (Number of items)"]

    # Filter out years depending on the time window
    
    years = list(range(window[0],window[1]))

    # y_all = data[years] # calculate the cumulative sum

    # year_range = range(int(window[0]), int(window[1]) + 1)

    years = [year for year in years if int(year) >= window[0] and int(year) <= window[1]] # filter out the years according to the window
    
    pred = forecast(data_domestic)


    y_domestic = []
    for year in years:
        if year in data_domestic.index:
            y_domestic.append(data_domestic[year])
        elif year in pred.index:
            y_domestic.append(pred[year])
        else:
            y_domestic.append(0)



    data_trade = flow_trade.loc[product, "Net Imports"]
    pred = forecast(data_trade)

    y_trade = []
    for year in years:
        if year in data_trade.index:
            y_trade.append(data_trade[year])
        elif year in pred.index:
            y_trade.append(pred[year])
        else:
            y_trade.append(0)


    flow_fig = go.Figure(data=[
        go.Bar(
            x=years,
            y=y_trade,
            name="Imports"
        ),
        
        go.Bar(
            x=years,
            y=np.round(y_domestic),
            name="domestic flow"),
        ])
    
    # Update the layout according to the requirements
    flow_fig.update_layout(
        title=f"<b>Inflow</b><br><sup>Cumulative Domestic Flow: {np.round(sum(y_domestic)/1_000_000,1)}M <br>Cumulative Imports: {np.round(sum(y_trade)/1_000_000,1)}M</sup>",
        yaxis_title="Number of items",
        margin=margins,
        # autosize=False,
        # width=800,
        # height=500,
        barmode='stack'
    )

    flow_fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
    ),
    height=300)

    return flow_fig # [html.P("Cummulated Number of products: ") , html.Strong(str(y_filtered.sum()/1_000_000)), html.Strong('M '), f"as of {max(years)}"]

In [9]:
def get_stock(shape=2, scale=9, product="Laptop"):
    

    product_data = storage_data.set_index("Year")[product].iloc[3:]

    
    product_data = product_data.loc[~pd.isna(product_data)]

    data_domestic = domestic_flow.loc[product, "Domestic", "Volume (Number of items)"]
    data_trade = flow_trade.loc[product, "Net Imports"]

    pred_domestic = forecast(data_domestic)
    pred_trade = forecast(data_trade)



    years = list(range(MIN_YEAR,MAX_YEAR))# list(storage["Year"]) + list(range(max(list(storage["Year"]))+1,2031))

    y_stock = []

    # print(product_data.index)

    for year in years:
        stock = 0
        if year in data_domestic.index:
            stock += data_domestic[year]
        if year in data_trade.index:
            stock += data_trade[year]
        if year not in data_trade.index and year not in data_domestic.index and year in product_data.index:
            stock = product_data[year]
        if year in pred_trade.index:
            stock += pred_trade[year]
        if year in pred_domestic.index:
            stock += pred_domestic[year]

            
        y_stock.append(stock)

    y_stock = pd.Series(y_stock)

    diff = list(y_stock.values)
    diff[0] = 0


    # diff[0] = product_data.iloc[0]

    # diff = diff + [0] * (2030-max(storage["Year"])) # 10

    x = np.arange(1,len(diff)+1)

    quants = stats.weibull_min.pdf(x, shape, 0, scale)

    losses = []

    for idx in range(MAX_YEAR-MIN_YEAR): # max(storage["Year"]) - min(storage["Year"])
        x = [diff[idx] * quant for quant in quants]
        loss = [0]*idx + x
        losses.append(loss[:len(diff)])


    # loss.insert(0,0)
    # losses.append(loss[:len(diff)])


    total_losses = pd.DataFrame(losses).sum(axis=0).values

    stock = []

    stock.append(diff[0] - total_losses[0])

    for i in range(1,len(total_losses)):
        stock.append(stock[-1] + diff[i] - total_losses[i])

    return years, stock, total_losses, quants
    return years, stock, total_losses, quants, y_stock, pd.DataFrame(losses), diff# pd.DataFrame(losses)# 


In [10]:
#********************************************************************************
# Defining the Layout of the application
#********************************************************************************

layout = html.Div([
    
    dbc.Row([
        
        # Adding the sidebar/navbar
        dbc.Col([sidebar], width=3),
        
        # The section for Tree plot and the EoL values
        dbc.Col([
        
            dbc.Row([
                
                # Adding tree plot
                dbc.Col([
                    dbc.Row(dcc.Graph(id="tree-fig")),
                    dbc.Row(dcc.Graph(id="weibull-fig")),

                ], width=6),# style={"border-right":"1px black solid"}),

                dbc.Col([
                    dbc.Row(dcc.Graph(id="imports-fig")),
                    dcc.Graph(id="stock-fig"),
                    dcc.Graph(id="outflow-fig")                    
                ], width=6),# style={"border-right":"1px black solid"}),
            ])
    ], width=9)
    ])
])

# *******************************************************************************
# Application Execution
#********************************************************************************

# Define the applicartoin and theme
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = layout # Add layout to the app


# Callback for interactively updating the charts and values

@app.callback(
    Output("weibull-shape", "value"),
    Output("weibull-scale", "value"),
    Input("navbar-product-dropdown", "value")
)

def update_weibull_sliders(product_selected):
    wb = product_data.loc[product_data["product_category(2)"] == product_selected][['Lifespan_Weibull_shape', 'Lifespan_Weibull_scale']].iloc[0].values
    shape = wb[0]
    scale = wb[1]

    return shape, scale

@app.callback(
    Output("stock-fig", "figure"),
    Output("median-lifetime", "children"),
    Output("outflow-fig", "figure"),
    Output("weibull-fig", "figure"),
    Input("weibull-shape", "value"),
    Input("weibull-scale", "value"),
    Input('navbar-interestwindow-slider', "value")
)

def update_stock_figure(shape, scale, window):
    years, stock, total_loss, quants = get_stock(shape,scale, "Laptop")

    years_filtered = []
    stock_filtered = []
    total_loss_filtered = []

    for idx, year in enumerate(years):
        if window[0] < year < window[1]:
            years_filtered.append(year)
            stock_filtered.append(stock[idx])
            total_loss_filtered.append(total_loss[idx])

    stock_fig = go.Figure(data=[
        go.Scatter(
            x=years_filtered, 
            y=stock_filtered,
            name="stock"
        ),
        
    ])

    outflow_fig = go.Figure(data=[
        go.Scatter(
            x=years_filtered, 
            y=total_loss_filtered,
            name="outflow"
        )
    ])

    weibull_fig = px.line(quants.cumsum())

    

    median_lifetime = scale * np.log(2) ** (1/shape)

    weibull_fig.update_layout(
        title="<b>Probability of Failure (CDF)</b>",
        yaxis_title="Probability",
        xaxis_title="Year",
        margin=margins,
        showlegend=False
        )

    stock_fig.update_layout(
        title=f"<b>Stock</b><br><sup>Cumulative: {round(np.sum(stock_filtered)/1_000_000, 1)}M</sup>",
        yaxis_title="Number of items",
        margin=margins,
        height=300
        )

    outflow_fig.update_layout(
        title=f"<b>Outflow</b><br><sup>Cumulative: {round(np.sum(total_loss_filtered)/1_000_000, 1)}M</sup>",
        yaxis_title="Number of items",
        margin=margins,
        height=300
        )

    return stock_fig, f"Median Lifetime: {np.round(median_lifetime,2)} Y", outflow_fig, weibull_fig


@app.callback(
    Output("eol-lower", "children"), # For the EoL Lower value
    Output("eol-upper", "children"), # For the EoL Upper value
    Output("eol-average", "children"), # For the EoL Average value
    # Output("imports-sum", "children"), # For the cumulative sum box
    Output("tree-fig", "figure"), # For the tree plot
    Output("imports-fig", "figure"), # For the empty plots
    Input('navbar-product-dropdown', "value"), # For product dropdown slider
    Input('navbar-interestwindow-slider', "value") # For Interest of window slider
    #Input('navbar-lifespan-slider', "value"), # This can be activated when necessary
)

def update_display(product_selected, window_selected):

    """
    Based on the callbacks set above, this functions returns updated values when
    interaction happens on the dashboard
    """

    eols = [str(i.values[0]) for i in get_eol(product_selected)] # Extract the selected product

    fig = get_flow(product_selected, "Domestic", window_selected) # Get the flow chart for the product selected above
    
    # The returns all objects for new selected product or changed window of interested
    return (
        "Lower end of Life: " + str(eols[0]), # EoL Upper value
        "Upper end of Life: " + str(eols[1]), # EoL Lower value
        "Average end of Life " + str(eols[2]), # EoL average value
        # total, # Sum of the bar plot
        get_treemap(product_selected), # Treemap for composition
        fig # The bar plot
    )

app.run_server(mode='external', port = 8055, dev_tools_ui=True, debug=True,
              use_reloader=False, threaded=True)

Dash is running on http://127.0.0.1:8055/

Dash app running on http://127.0.0.1:8055/
